<a href="https://colab.research.google.com/github/lucaszm7/DPVC-MNIST/blob/main/Copy_of_MNIST_Tutorial_T1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Defining and Training a simple Network for classifying digits using the MNIST dataset**

Import statements for using the **TensorFlow** framework and the **Keras** API

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import numpy as np


Load the **MNIST dataset** in Keras, both train and test images and labels.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Check the shape of the train images and the number of labels, and inspect the labels and one training image

In [ ]:
import matplotlib.pyplot as plt
#
#   Check the shape of the train images and the number of labels
#
print('The shape of the traing dataset: ', train_images.shape)
print('The number of training labels is ', len(train_labels))
print('Training labels: ', train_labels)
#
#   display a given training image. Index starts at 0 (zero)
#
img_index = 2
digit = train_images[img_index]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()


Define a simple architecture (model) consisting of 2 dense (fully connected) layers. The first layer uses **ReLU**, while the second one uses **softmax** as activation function.

In [ ]:
model = keras.Sequential([
  layers.Dense(512, activation="relu"),
  layers.Dense(10, activation="softmax")
])

**To make the model ready for training**, we need to pick three more things as part of the compilation step:

1) **An optimizer** — The mechanism through which the model will update itself based on the training data it sees, so as to improve its performance;

2) **A loss function** — How the model will be able to *measure its performance on the training data*, and thus how it will be able to steer itself in the right direction;

3) **Metrics to monitor during training and testing** — Here, we’ll only care about accuracy (the fraction of the images that were correctly classified).

In [ ]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

Before training, we’ll preprocess the data by reshaping it into the shape the model expects and scaling it so that all values are in the [0, 1] interval. Previously, our (MNIST) training images were stored in an array of shape (60000, 28, 28) of type uint8 with values in the [0, 255] interval. We’ll transform it into a float32 array of shape (60000, 28*28) with values between 0 and 1.

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

We are now ready to **train the model**, which actually means **fit the model to its training data**. For this, we **need to specify** the *training images and labels* (this is a supervised learning), as well as the **number of epochs**, and the **batch size**.

The **number of epochs** specifies the number of times the training dataset will be fully visited.

The **batch size** specifies the number of samples (images) that will be processed before the weights of the network are updated.



In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

We can check the architecture of our network and the number of associated trainable parameters. Since the model has not been initialized, we will need to build it first. If, instead, we had already trained it, there would be no need to build it.

In [ ]:
model.summary()

The **number of trainable parameters** in the network above (407,050) is obtained in the following way: the input layer is a vector with 28x28 = 784 elements. The first (dense) layer has 512 neurons. Fully connecting these elements results in 784x512 = 401,408 weights + 512 bias = 401,920 parameters.  Fully connecting the 512 elements from the first layer to the 10 elements of the second layer requires 512x10 + 10 = 5,130 parameters. The grand total is then 401,920 + 5,130 = 407,050.   

Evaluate the model on the test_images, obtaining average loss and accuracy values

In [ ]:
model.evaluate(test_images, test_labels, batch_size=128)

Check the result for any particular example

In [ ]:
img_index = 12
test = test_images[img_index]
prediction = model.predict(np.array([test,]))
print(prediction)
print('Predicted digit: ', np.argmax(prediction),'. True value: ', test_labels[img_index])


**Experimenting with some hyperparameters** as part of the suggested exercises.   

In [ ]:
model3 = keras.Sequential([
  layers.Dense(512, activation="relu"),
#    layers.Dense(25, activation="sigmoid"),
  layers.Dense(10, activation="softmax")
])

model3.compile(optimizer="rmsprop",
#model3.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model3.fit(train_images, train_labels, epochs=5, batch_size=128)

model3.summary()

model3.evaluate(test_images, test_labels, batch_size=128)

# The Manifold Hypothesis

**Extracted from the Book: Deep Learning with Python**

The input to an MNIST classifier (before preprocessing) is a 28 × 28 array of integers between 0 and 255. The total number of possible input values is thus 256 to the power of 784—much greater than the number of atoms in the universe. However, very few of these inputs would look like valid MNIST samples: **actual handwritten digits only occupy a tiny subspace of the parent space of all possible 28 × 28 uint8 arrays**. What’s more, this subspace isn’t just a set of points sprinkled at random in the parent space: **it is highly structured**.

First, *the subspace of valid handwritten digits is continuous*: if you take a sample and modify it a little, it will still be recognizable as the same handwritten digit. Further, *all samples in the valid subspace are connected by smooth paths that run through the subspace*. This means that if you take two random MNIST digits A and B, there exists a sequence of “intermediate” images that morph A into B, such that two consecutive digits are very close to each other (see figure 5.7). Perhaps there will be a few ambiguous shapes close to the boundary between two classes, but even these shapes would still look very digit-like.

In technical terms, you would say that **handwritten digits form a manifold within the space of possible 28 × 28 uint8 arrays**. That’s a big word, but the concept is pretty intuitive. **A manifold is a lower-dimensional subspace of some parent space that is locally similar to a linear (Euclidian) space**.

More generally, **the manifold hypothesis posits that all natural data lies on a low-dimensional manifold within the high-dimensional space where it is encoded**. That’s a *pretty strong statement about the structure of information in the universe*. As far as we know, it’s accurate, and **it’s the reason why deep learning works. It’s true for MNIST digits, but also for human faces, tree morphology, the sounds of the human voice, and even natural language.**

**The manifold hypothesis implies that**
- **Machine learning models only have to fit relatively simple, low-dimensional,
highly structured subspaces within their potential input space (latent manifolds)**.

- **Within one of these manifolds, it’s always possible to interpolate between two inputs**, that is to say, morph one into another via a continuous path along which
all points fall on the manifold.

In [ ]:
from google.colab import drive
drive.mount('/content/Data')
from IPython.display import Image

Image(filename='Data/MyDrive/Colab_DL_folder/DL_images/Table_performance_MNIST_Lab_1.PNG',width=800,height=300)

# **EXERCISES**

**1)** Save a copy of the loss and accuracy values from all epochs from the original training (e.g., copy and paste them on a notepad) and test. Then train (fit) the model again for 5 epochs and compare the obtained training loss and accuracy values with the original ones. What do you observe? Train it one more time for 5 epochs. What do you conclude?

**2)** Now, compile and build the model again before training it for 5 epochs. What do you observe with respect to the training loss and accuracy?

**3)** Did the test loss and accuracy values improve? (Do not forget to save the previous values before re-evaluating the model). What did you conclude?

**4)** Now, define another model (model2) with exactly the same architecture. Then, compile, build, and train it. What do you observe with respect to the training loss and accuracy values?

**5)** Now, define another model (model3) with exactly the same architecture. Then, compile, build, and train it for 15 epochs. What do you observe with respect to the training loss and accuracy values? and the test loss and accuracy values?

**6)** Change the number of neurons in the first hidden layer of the network above from 512 to 25. Check the number of trainable parameters. Compile, train it for 5 epochs and evaluate the new network. Compare its performance with the original one. How about its test loss and accuracy values? Compare the number of parameters between this and the original network.

**7)** Train this new network for additional 5 epochs and then to additional 5 epochs. What do you observe with respect to the training loss and accuracy values?  How about its test loss and accuracy values? What do you conclude?

**8)** Change the number of neurons in the first hidden layer of the network above to 256. Check the number of trainable parameters. Compile, train it for 5 epochs and evaluate the new network. Compare its performance with the original one. How about its test loss and accuracy values? Compare the number of parameters between this and the previous network (25 neurons in the first layer). Does its performance agree with your expectation?

**9)** Change the number of neurons in the first layer of the network above back to 512. Compile, train it for 5 epochs and evaluate the new network. Compare its performance with the original one. How about its test loss and accuracy values? Compare the number of parameters between this and the previous network. Does its performance agree with your expectation?

**10)** Now, go on the *Runtime* tab and select the operation *Restart and run all* to reset all variables. Retrain the last newtork, What do you observe?

**11)** Now set the optimizer to *"adam"* in the *model.compile* command, reset the variables again, and retrain your model. What do you observe with respect to its performance?

**12)** Now replace "relu" with "sigmoid" in the activation function of the first dense layer of the network. What do you observe?

